In [ ]:
from tensorflow.python.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, Flatten, Dropout, Dense
from tensorflow.python.keras.models import Model
import numpy as np


import tensorflow as tf
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.callbacks import History
from lib.KerasHelpers.modelhelpers import model_placement
from src.models.model import TextCNN
from argparse import ArgumentParser
import deepdish as dd
import numpy as np
import os

    
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from sklearn.model_selection import train_test_split



In [ ]:
# sequential implementaion (mulit-label classification)

def TextCNN():  
    model = Sequential()
    model.add(Embedding(1000,
                        20,
                        input_length=max_len))
    model.add(Dropout(0.2))
    model.add(Conv1D(256,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(0.2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    return model

In [ ]:
model = TextCNN()
# model.summary()
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.binary_accuracy])

In [ ]:
# chainwise implementation

def TextCNN(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM, num_classes, num_words, dropout_rate=0.4, flag='rand', embedding_weights=None):

    x_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

    embedding_layer = Embedding(input_dim=num_words,
                                output_dim=EMBEDDING_DIM,
                                embeddings_initializer='uniform',
                                input_length=MAX_SEQUENCE_LENGTH)
    x = embedding_layer(x_input)

    kernel_sizes = [3, 4, 5]
    pooled = []

    for kernel in kernel_sizes:

        conv = Conv1D(filters=100,
                      kernel_size=kernel,
                      padding='valid',
                      strides=1,
                      kernel_initializer='he_uniform',
                      activation='relu')(x)
        
        pool = MaxPooling1D(pool_size=MAX_SEQUENCE_LENGTH - kernel + 1)(conv)

        pooled.append(pool)

    merged = Concatenate(axis=-1)(pooled)

    flatten = Flatten()(merged)

    drop = Dropout(rate=dropout_rate)(flatten)
    
    x_output = Dense(num_classes, kernel_initializer='he_uniform', activation='softmax')(drop)

    return Model(inputs=x_input, outputs=x_output)

In [ ]:
x_train = 
# (5000, 128)

y_train = 
# (5000, 2)

drop_rate = 0.2
batch_size   = 32
num_epochs   = 12
lr = 0.001

kwargs = {'MAX_SEQUENCE_LENGTH': x_train.shape[1],
          'EMBEDDING_DIM': 300
          'num_classes': y_train.shape[1],
          'num_words': data_dict['num_words'],
          'dropout_rate': dropout_rate,
         }

model = TextCNN(**kwargs)

# model = model_placement(text_model, num_gpus=options.num_gpus)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=SGD(lr=lr))

model.fit(x=x_train, y=y_train, 
          batch_size=batch_size, 
          epochs=num_epochs, 
          validation_data=validation_data)